In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [30]:
# load data
train = pd.read_csv("output/random_train0310.csv")
test = pd.read_csv("output/random_test0310.csv")
val = pd.read_csv("output/random_val0310.csv")

train = train[train.LoanStatus != "MISSING"]
train = train[train.LoanStatus == "CHGOFF"]
test = test[test.LoanStatus != "MISSING"]
test = test[test.LoanStatus == "CHGOFF"]
val = val[val.LoanStatus != "MISSING"]
val = val[val.LoanStatus == "CHGOFF"]

# filtering for the columns you don't want to include for your model
c = train.columns.tolist()
dropped_columns = ['LoanStatus', 'ChargeOffDate', 'GrossChargeOffAmount', 'BorrZip', 'CDC_Zip', 'BorrCity',
           'CDC_City', 'ThirdPartyLender_City', 'ProjectCounty', 'ApprovalDate']
for col in dropped_columns:
c.remove(col)

# seperate data into covariates and targets
x_train = train[c]
x_test = test[c]
x_val = val[c]

y_train = train.GrossChargeOffAmount.values/train.GrossApproval.values
y_test = test.GrossChargeOffAmount.values/test.GrossApproval.values
y_val = val.GrossChargeOffAmount.values/val.GrossApproval.values

x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
x_val = pd.get_dummies(x_val)

def add_missing_columns(d1, d2):
    missing_cols_2 = (set(d1.columns) - set(d2.columns))
    missing_cols_1 = (set(d2.columns) - set(d1.columns))
    for c in missing_cols_2:
        d2[c] = 0
    for c in missing_cols_1:
        d1[c] = 0
return d1, d2

x_train, x_test = add_missing_columns(x_train, x_test)
x_train, x_val = add_missing_columns(x_train, x_val)
x_test, x_val = add_missing_columns(x_test, x_val)

print x_train.shape
print y_train_norm.shape
print train.dtypes

BorrCity                         object
BorrState                        object
BorrZip                          object
CDC_City                         object
CDC_State                        object
CDC_Zip                          object
ThirdPartyLender_City            object
ThirdPartyLender_State           object
ApprovalDate                     object
ApprovalFiscalYear                int64
DeliveryMethod                   object
subpgmdesc                       object
TermInMonths                      int64
ProjectCounty                    object
ProjectState                     object
BusinessType                     object
LoanStatus                       object
ChargeOffDate                    object
GrossChargeOffAmount            float64
SP500_Yearly_Return             float64
CPI                             float64
Log_GrossApproval_Norm          float64
Log_HPI_Norm                    float64
ThirdPartyDollars_Norm          float64
TermMultipleYear                   bool


In [22]:
# do a normal linear regression first
linear = LinearRegression()
print(linear.summary())

                             OLS Regression Results                             
Dep. Variable:     GrossChargeOffAmount   R-squared:                       0.732
Model:                              OLS   Adj. R-squared:                  0.724
Method:                   Least Squares   F-statistic:                     89.58
Date:                  Wed, 14 Mar 2018   Prob (F-statistic):               0.00
Time:                          18:57:14   Log-Likelihood:                -5414.7
No. Observations:                  7124   AIC:                         1.125e+04
Df Residuals:                      6912   BIC:                         1.271e+04
Df Model:                           211                                         
Covariance Type:              nonrobust                                         
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [26]:
regularized = smf.OLS(y_train_norm, x_train.astype(float)).fit_regularized(method='elastic_net',alpha=0.01,L1_wt=0.0,start_params=linear.params)
print(regularized.summary())

None
